In [1]:
%load_ext autoreload
%autoreload 2
import datetime
import numpy as np
import os
import pandas as pd
import sys

project_dir = os.sep.join([os.getcwd(), '..'])
#sys.path.append(os.sep.join([project_dir, 'src', 'processed']))
processed_data_dir = os.sep.join([project_dir, 'data', 'processed'])

%matplotlib inline

In [2]:
df_met  = pd.read_feather(os.sep.join([processed_data_dir, 'blindern_met_features.feather']))
df_date = pd.read_feather(os.sep.join([processed_data_dir, 'datetime_features.feather']))

In [3]:
df_met.head()

,level_0,index,temperature,wind_speed,humidity,sunshine,precipitation,wind_direction_cat,weather_fair/cloudy,weather_fog/haze,weather_thunderstorm,weather_rain,weather_snow,weather_other
0,0,2016-04-01 00:00:00+00:00,2.1,6.0,46.0,0.0,0.0,7,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2016-04-01 01:00:00+00:00,2.2,5.7,45.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,2016-04-01 02:00:00+00:00,0.7,5.8,51.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,2016-04-01 03:00:00+00:00,-0.3,2.0,55.0,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0
4,4,2016-04-01 04:00:00+00:00,-1.8,2.2,62.0,0.0,0.0,5,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_date.head()

,DateTime,hour,weekday,month,public_holiday,school_holiday
0,2016-04-01 00:00:00+00:00,0,4,4,0,0
1,2016-04-01 00:05:00+00:00,0,4,4,0,0
2,2016-04-01 00:10:00+00:00,0,4,4,0,0
3,2016-04-01 00:15:00+00:00,0,4,4,0,0
4,2016-04-01 00:20:00+00:00,0,4,4,0,0


In [5]:
df_joined = pd.merge(df_met, df_date, left_on='index', right_on='DateTime', how='right')
df_joined.drop(columns=['level_0', 'index'], inplace=True)

In [6]:
df_joined.head()

,temperature,wind_speed,humidity,sunshine,precipitation,wind_direction_cat,weather_fair/cloudy,weather_fog/haze,weather_thunderstorm,weather_rain,weather_snow,weather_other,DateTime,hour,weekday,month,public_holiday,school_holiday
0,2.1,6.0,46.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,2016-04-01 00:00:00+00:00,0,4,4,0,0
1,2.2,5.7,45.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2016-04-01 01:00:00+00:00,1,4,4,0,0
2,0.7,5.8,51.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2016-04-01 02:00:00+00:00,2,4,4,0,0
3,-0.3,2.0,55.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2016-04-01 03:00:00+00:00,3,4,4,0,0
4,-1.8,2.2,62.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,2016-04-01 04:00:00+00:00,4,4,4,0,0
